# Load the data

In [32]:
import pandas as pd
import pickle
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

data_dir = "../../data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
# load data
Labeled_data = load_obj("Labeled_data")

In [43]:
count_positive = 0
count_negative = 0
positive_label=[]
negative_label=[]
count=0
for label_data in Labeled_data:
    if (count_positive+count_negative)>=500000:
        break
    if label_data[1]==1:
        if count_positive >= 250000:
            continue
        positive_label.append(label_data)
        count_positive += 1
    elif label_data[1]==0:
        count_negative += 1
        negative_label.append(label_data)
# print(count_positive)
# print(count_negative)
label_5w = positive_label +negative_label
len(label_5w)

250000
250000


500000

In [44]:
X=[]
y=[]
# get the data and label
Labeled_data[0]
for instance in label_5w:
    row = [instance[2],instance[3],instance[4],instance[5]]
    y.append(instance[1])
    X.append(row)
X=pd.DataFrame(X)

In [21]:
X.shape

(50000, 4)

In [45]:
from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)
X_train, X_validation, y_train, y_validation  = train_test_split(X_t,y_t)

In [8]:
import pandas as pd
training_df = pd.DataFrame([1,2,3,4]).T
training_df= training_df.append(pd.DataFrame([1,2,3,4]).T)
training_df


0  1  2  3
0  1  2  3  4
0  1  2  3  4

In [ ]:
# second test
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, cross_val_score
rf = RandomForestClassifier()
X_train = X_t
y_train = y_t

pipeline = Pipeline([
       ('clf', RandomForestClassifier(criterion='entropy'))
   ])
parameters = {
       'clf__n_estimators': (50, 100, 500),
       'clf__max_depth': (10, 50, 100),
       'clf__min_samples_split': (100, 1000, 10000),
       'clf__min_samples_leaf': (10, 100, 1000, 10000)
}
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,
   verbose=1, scoring='roc_auc', cv=3)

grid_search.fit(X_train, y_train)
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
predictions = grid_search.predict(X_test)
print('Accuracy:', accuracy_score(y_test, predictions))
print('Precision:', precision_score(y_test, predictions))
print('Recall:', recall_score(y_test, predictions))
# print('roc_auc_score:', roc_auc_score(y_test, predictions))

# Conclusion
# clf__n_estimators : more esitmators
# clf__max_depth: more depth
# clf__min_samples_split: less split
# clf__min_samples_leaf: less leaf

In [33]:
print('roc_auc_score:', roc_auc_score(y_test, predictions))

roc_auc_score: 0.9268691536328261


In [36]:
BasicFeatures = load_obj("pre_features")
pre_features = BasicFeatures
# XUDONG
#Adamic-Adar similarity
def AA(Node1, Node2):
    sim = 0.0
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    #print(len(common_neighors))
    for node in common_neighors:
        sim += pre_features[node][1]
    return sim

#Jaccard
def Jaccard(Node1, Node2):
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    lm = len(common_neighors)
    if lm == 0:
        return 0
    else:
        return (0.0+lm)/(n1[0] + n2[0] -lm)

#Cosine
def Cosine(Node1, Node2):
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    lm = len(common_neighors)
    if lm == 0:
        return 0
    else:
        return (0.0+lm)/((n1[0])*(n2[0]))    
    
#Salton Similarity
def salton_similarity(node1, node2):
    n1 = pre_features[node1]
    n2 = pre_features[node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    inter = len(common_neighors)
    degree_out_flow = n1[6]
    degree_in_flow = n2[4]
    if inter == 0:
        return 0
    else:
        sqrt_of_degree = math.sqrt(degree_out_flow * degree_in_flow)
        salton = inter / sqrt_of_degree
        probability = 1 /(1 - math.log(salton)*0.2)
        return probability

In [40]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
predictions = grid_search.predict(X_test)
accuracy_score(y_test, predictions)

0.92672

In [37]:
# Save the result to 
from tqdm import tqdm
with open(data_dir + "test-public.txt", "r") as f:
     test_data = f.readlines()
test_data = [i.split() for i in test_data[1:]]
def predict():
    """
    make the prediction using the jaccard's coefficient
    """
    result = []
    for line in tqdm(test_data):
        # converse to integer
        node_x = int(line[1].strip())
        node_y = int(line[2].strip())
#         score = get_resource_allocation(node_x, node_y)

        score_AA = AA(node_x,node_y)
        # jc
        score_Jaccard = Jaccard(node_x,node_y)
        # cos
        score_Cosine = Cosine(node_x,node_y)
        # salton
        score_salton_similarity = salton_similarity(node_x,node_y)
        X_test = pd.DataFrame([score_AA, score_Jaccard, score_Cosine, score_salton_similarity]).T
#         print(X_test)
        single_result = grid_search.predict(X_test)[0]
        result.append((line[0], single_result))
#         print(result)
    return result
result = predict()

100%|██████████| 2000/2000 [01:21<00:00, 24.52it/s]


In [38]:
print(result)

[('1', 0), ('2', 1), ('3', 0), ('4', 1), ('5', 1), ('6', 1), ('7', 0), ('8', 0), ('9', 1), ('10', 0), ('11', 1), ('12', 0), ('13', 0), ('14', 0), ('15', 1), ('16', 0), ('17', 1), ('18', 0), ('19', 0), ('20', 1), ('21', 0), ('22', 0), ('23', 0), ('24', 1), ('25', 0), ('26', 1), ('27', 0), ('28', 1), ('29', 1), ('30', 1), ('31', 0), ('32', 0), ('33', 1), ('34', 1), ('35', 0), ('36', 1), ('37', 0), ('38', 0), ('39', 0), ('40', 0), ('41', 1), ('42', 1), ('43', 0), ('44', 0), ('45', 1), ('46', 0), ('47', 0), ('48', 1), ('49', 1), ('50', 1), ('51', 1), ('52', 1), ('53', 1), ('54', 1), ('55', 0), ('56', 1), ('57', 0), ('58', 0), ('59', 1), ('60', 1), ('61', 1), ('62', 0), ('63', 1), ('64', 0), ('65', 0), ('66', 1), ('67', 1), ('68', 0), ('69', 1), ('70', 0), ('71', 0), ('72', 1), ('73', 0), ('74', 1), ('75', 1), ('76', 0), ('77', 1), ('78', 1), ('79', 1), ('80', 1), ('81', 0), ('82', 1), ('83', 1), ('84', 0), ('85', 0), ('86', 0), ('87', 0), ('88', 1), ('89', 1), ('90', 0), ('91', 0), ('92', 

In [41]:
import csv
import time
'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())

"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
    headers = ['id','Prediction']

    with open(filename + str(nowtime()) + ".csv", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(result)
save_prediction_to_csv(result, "shawn_rf_second_")

# Explantory

# Modelling

# Please save the training set as the csv file.

# Prediction